In [52]:
from dimod import ConstrainedQuadraticModel, Real, Integer, quicksum

def mps2cmq(filename):
    ### read the mps file ###
    mpsFile = open(filename, "r")
    mpsLine = mpsFile.readlines()
    mpsLine = [s.replace('\n', '') for s in mpsLine]

    readRow, readCol, readInt, readRHS, readBnd, readObj = False, False, False, False, False, False
    equations, int_variables, real_variables, objEq = {}, set(), set(), None
    LHSeq, RHSeq, upBound, lowBound = {}, {}, {}, {}

    for i in range(len(mpsLine)):
        string = mpsLine[i]  
        if  string == 'ROWS':
            readRow = True
        elif string == 'COLUMNS':
            readRow = False
            readCol = True
            LHSeq = {k: [] for k in equations.keys()}
            RHSeq = {k: [] for k in equations.keys()}
        elif string == 'RHS':
            readCol = False
            readRHS = True
        elif string == 'BOUNDS':
            readRHS = False
            readBnd = True

        if readRow and string != 'ROWS':
            relation, eq = string.split()
            equations[eq] = relation
            if not readObj and relation == 'N': ObjEq = eq
        
        if readCol and string != 'COLUMNS':
            strList = string.split()
            if strList[1] == "\'MARKER\'" and strList[2] == "\'INTORG\'":
                readInt = True
                continue
            elif strList[1] == "\'MARKER\'" and strList[2] == "\'INTEND\'":
                readInt = False
                continue

            var = strList[0]
            if readInt:
                int_variables.add(var)
            else:
                real_variables.add(var)
            for i in range(0, len(strList)//2):
                LHSeq[strList[i*2+1]].append((var, strList[i*2+2]))
        
        if readRHS and string != 'RHS':
            strList = string.split()
            for i in range(0, len(strList)//2):
                RHSeq[strList[i*2+1]] = strList[i*2+2]
        
        if readBnd and string != 'BOUNDS':
            strList = string.split()
            if strList[0] == 'UP':
                upBound[strList[2]] = strList[3]
            elif strList[0] == 'LO':
                lowBound[strList[2]] = strList[3]

    mpsFile.close()

    ### convert into cqm ###
    cqm = ConstrainedQuadraticModel()

    # variables
    x = {v: Real(v) for v in real_variables}
    x_int = {v: Integer(v) for v in int_variables}
    x.update(x_int)

    # the objective function and constraint equations
    for eq in equations:
        currLHS = quicksum(x[v]*float(coef) for v, coef in LHSeq[eq])

        if eq == ObjEq:
            cqm.set_objective(currLHS)
        
        if equations[eq] == 'E':
            cqm.add_constraint(currLHS == float(RHSeq[eq]), label=eq)
        elif equations[eq] == 'G':
            cqm.add_constraint(currLHS >= float(RHSeq[eq]), label=eq)
        elif equations[eq] == 'L':
            cqm.add_constraint(currLHS <= float(RHSeq[eq]), label=eq)

    # constraints on the variables
    for k in upBound.keys():
        cqm.set_upper_bound(k, float(upBound[k]))
    for k in lowBound.keys():
        cqm.set_lower_bound(k, float(lowBound[k]))
    
    return cqm

In [56]:
from dwave.system import LeapHybridCQMSampler

# read mps file
cqm1 = mps2cmq('markshare_4_0.mps')

In [45]:
# run the solver
sampler = LeapHybridCQMSampler()
sampleset = sampler.sample_cqm(cqm1, label='markshare_4_0')

In [46]:
# analyze the results
feasible_sampleset = sampleset.filter(lambda d: d.is_feasible)

In [54]:
sampleset.first.energy

0.0